<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_human_feedback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Task Solving with Code Generation, Execution, Debugging & Human Feedback

AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

In this notebook, we demonstrate how to use `AssistantAgent` and `UserProxyAgent` to solve a challenging math problem with human feedback. Here `AssistantAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for a user to execute the code written by `AssistantAgent`. By setting `human_input_mode` properly, the `UserProxyAgent` can also prompt the user for feedback to `AssistantAgent`. For example, when `human_input_mode` is set to "ALWAYS", the `UserProxyAgent` will always prompt the user for feedback. When user feedback is provided, the `UserProxyAgent` will directly pass the feedback to `AssistantAgent`. When no user feedback is provided, the `UserProxyAgent` will execute the code written by `AssistantAgent` and return the execution results (success or failure and corresponding outputs) to `AssistantAgent`.

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [1]:
%pip install pyautogen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [2]:
import autogen

config_list = autogen.config_list_from_json("OAI_CONFIG_LIST")

In [4]:
config_list = [
    {
        'model': 'gpt-3.5-turbo',
        'api_key': 'sk-xxxxxxxxxxxxx',
    }
]

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST".

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },  # OpenAI API endpoint for gpt-4
    {
        'model': 'gpt-4',
        'api_key': '<your first Azure OpenAI API key here>',
        'api_base': '<your first Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },  # Azure OpenAI API endpoint for gpt-4
    {
        'model': 'gpt-4',
        'api_key': '<your second Azure OpenAI API key here>',
        'api_base': '<your second Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },  # another Azure OpenAI API endpoint for gpt-4
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your OpenAI API key here>',
    },  # OpenAI API endpoint for gpt-3.5-turbo
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your first Azure OpenAI API key here>',
        'api_base': '<your first Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },  # Azure OpenAI API endpoint for gpt-3.5-turbo
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your second Azure OpenAI API key here>',
        'api_base': '<your second Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },  # another Azure OpenAI API endpoint for gpt-3.5-turbo
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choosing "upload file" icon.

You can set the value of config_list in other ways you prefer, e.g., loading from a YAML file.

## Construct Agents

We construct the assistant agent and the user proxy agent.

In [5]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "seed": 41,
        "config_list": config_list,
    }
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
)

# the purpose of the following line is to log the conversation history
autogen.ChatCompletion.start_logging()


## Perform a task

We invoke the `initiate_chat()` method of the user proxy agent to start the conversation. When you run the cell below, you will be prompted to provide feedback after receiving a message from the assistant agent. If you don't provide any feedback (by pressing Enter directly), the user proxy agent will try to execute the code suggested by the assistant agent on behalf of you, or terminate if the assistant agent sends a "TERMINATE" signal at the end of the message.

In [6]:
math_problem_to_solve = """
Find $a + b + c$, given that $x+y \\neq -1$ and
\\begin{align}
	ax + by + c & = x + 7,\\
	a + bx + cy & = 2x + 6y,\\
	ay + b + cx & = 4x + y.
\\end{align}.
"""

# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(assistant, message=math_problem_to_solve)

user_proxy (to assistant):


Find $a + b + c$, given that $x+y \neq -1$ and
\begin{align}
	ax + by + c & = x + 7,\
	a + bx + cy & = 2x + 6y,\
	ay + b + cx & = 4x + y.
\end{align}.


--------------------------------------------------------------------------------
assistant (to user_proxy):

To find the values of $a$, $b$, and $c$, we can solve the system of equations using linear algebra. 

Let's rewrite the system of equations in matrix form:

\begin{align*}
\begin{bmatrix}
x & y & 1 \\
1 & x & y \\
y & 1 & x \\
\end{bmatrix}
\begin{bmatrix}
a \\
b \\
c \\
\end{bmatrix}
&=
\begin{bmatrix}
x + 7 \\
2x + 6y \\
4x + y \\
\end{bmatrix}
\end{align*}

Now, we can solve for the values of $a$, $b$, and $c$ by solving the matrix equation:

\begin{align*}
\mathbf{A}\mathbf{x} = \mathbf{b}
\end{align*}

where
$\mathbf{A} = \begin{bmatrix}
x & y & 1 \\
1 & x & y \\
y & 1 & x \\
\end{bmatrix}$,
$\mathbf{x} = \begin{bmatrix}
a \\
b \\
c \\
\end{bmatrix}$, and
$\mathbf{b} = \begin{bmatrix}
x + 7 \\
2

user_proxy (to assistant):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "", line 5, in <module>
    A = np.array([[x, y, 1], [1, x, y], [y, 1, x]])
NameError: name 'np' is not defined


--------------------------------------------------------------------------------
assistant (to user_proxy):

I apologize for the oversight. It seems that the numpy library (`numpy`) was not imported before using it in the code. Let me correct that for you. Please execute the updated code below:

```python
# filename: solve_matrix_equation.py
import numpy as np

x = 3
y = 2

A = np.array([[x, y, 1], [1, x, y], [y, 1, x]])
b = np.array([x + 7, 2 * x + 6 * y, 4 * x + y])

x = np.linalg.solve(A, b)

a = x[0]
b = x[1]
c = x[2]

result = a + b + c

print("The sum of a, b, and c is:", result)
```

Please replace `3` and `2` with the appropriate values for `x` and `y` in your case. After running the code, it should provide the desired sum of `a`, `b`, and `c`. Let me k

user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
The sum of a, b, and c is: 7.0


--------------------------------------------------------------------------------
assistant (to user_proxy):

Great! I'm glad to hear that the code executed successfully and provided the expected output. The sum of `a`, `b`, and `c` is `7.0`.

If you have any more questions or need further assistance, feel free to ask.

--------------------------------------------------------------------------------
Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
user_proxy (to assistant):



--------------------------------------------------------------------------------
assistant (to user_proxy):

It appears that you have no further questions or tasks at the moment. If you have any more questions or need assistance in the future, feel free to ask. Have a great day!

-

## Analyze the conversation

The human user can provide feedback at each step. When the human user didn't provide feedback, the code was executed. The executed results and error messages are returned to the assistant, and the assistant is able to modify the code based on the feedback. In the end, the task is complete and a "TERMINATE" signal is sent from the assistant. The user skipped feedback in the end and the conversation is finished.

After the conversation is finished, we can save the conversation log between the two agents. The log can be accessed from `autogen.ChatCompletion.logged_history`.

In [7]:
print(autogen.ChatCompletion.logged_history)

{'[{"content": "You are a helpful AI assistant.\\nSolve tasks using your coding and language skills.\\nIn the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.\\n    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.\\n    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.\\nSolve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.\\nWhen using code, you must indicate the script type in the code block. The user c

In [8]:
import json

json.dump(autogen.ChatCompletion.logged_history, open("conversations.json", "w"), indent=2)